In [1]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
ABSOLUTE_PATH = "../../datasets/"

In [ ]:
df = pd.read_csv(ABSOLUTE_PATH+"flipkart-data.csv")
df = df[0:10000]


In [ ]:
def convert_to_arr(row):
    s = row["product_category_tree"].strip('[]').strip('""')
    s = s.split(">>")
    
    for word in s:
        word = word.strip()
    return s

In [ ]:
df["category_array"] = df.apply(convert_to_arr,axis=1)

In [ ]:
def get_keywords(row):
    sentence = str(row["description"])
    arr = process_keywords(sentence)
    cat_arr = row["category_array"]
    for word in cat_arr:
        arr = arr + process_keywords(word)
    return arr

In [ ]:
def process_keywords(sentence):
    sentence = sentence.lower()
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    words = tokenizer.tokenize(sentence)
    stop_words = set(stopwords.words('english')) 
    keywords = [w for w in words if not w in stop_words]
    return keywords
    

In [ ]:
df["keywords_array"] = df.apply(get_keywords,axis=1)


In [ ]:
df.drop("product_category_tree",inplace=True,axis=1)
df.drop("description",inplace=True,axis=1)

In [ ]:
df.to_csv(ABSOLUTE_PATH+"category_dataset.csv")

In [4]:
products = pd.read_csv(ABSOLUTE_PATH+"category_dataset.csv")

In [5]:
products = products[products.keywords_array.notnull()]

In [6]:
from gensim.corpora.dictionary import Dictionary

In [7]:
keywords = products["keywords_array"].tolist()
#keywords = [keyword.split() for keyword in keywords]

In [8]:
dictionary = Dictionary(keywords)

TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in keywords] 


In [ ]:
from gensim.models.tfidfmodel import TfidfModel
tfidf = TfidfModel(corpus) #create tfidf model of the corpus

In [ ]:
from gensim.similarities import MatrixSimilarity
# Create the similarity data structure. This is the most important part where we get the similarities between the movies.
sims = MatrixSimilarity(tfidf[corpus], num_features=len(dictionary))

In [ ]:
def keywords_recommendation(keywords, number_of_hits):
    query_doc_bow = dictionary.doc2bow(keywords) # get a bag of words from the query_doc
    query_doc_tfidf = tfidf[query_doc_bow] #convert the regular bag of words model to a tf-idf model where we have tuples
    # of the movie ID and it's tf-idf value for the movie

    similarity_array = sims[query_doc_tfidf] # get the array of similarity values between our movie and every other movie. 
    #So the length is the number of movies we have. To do this, we pass our list of tf-idf tuples to sims.

    similarity_series = pd.Series(similarity_array.tolist(), index=products.product_name.values) #Convert to a Series
    top_hits = similarity_series.sort_values(ascending=False)[:number_of_hits] #get the top matching results, 
    # i.e. most similar movies

    # Print the top matching movies
    print("Our top %s most similar products for the keywords %s are:" %(number_of_hits, keywords))
    for idx, (product,score) in enumerate(zip(top_hits.index, top_hits)):
        print("%d '%s' with a similarity score of %.3f" %(idx+1, product, score))

In [ ]:
keywords_test = ['shoes']
keywords_recommendation(keywords_test, 5)